In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import mlflow
import mlflow.tensorflow
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix

In [2]:
# Data generators with InceptionV3 expected input size
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_generator = train_datagen.flow_from_directory(
    'D://PROJECTS/ALL_PROJECTS/FREEZING_AND_NON_FREEZING/wavelet_training/data/train',
    target_size=(299, 299),  # Adjusted for InceptionV3
    batch_size=32,
    class_mode='binary'
)

validation_generator = valid_datagen.flow_from_directory(
    'D://PROJECTS/ALL_PROJECTS/FREEZING_AND_NON_FREEZING/wavelet_training/data/valid',
    target_size=(299, 299),  # Adjusted for InceptionV3
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    'D://PROJECTS/ALL_PROJECTS/FREEZING_AND_NON_FREEZING/wavelet_training/data/test',
    target_size=(299, 299),  # Adjusted for InceptionV3
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


Found 3583 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [11]:
# Using InceptionV3
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
for layer in base_model.layers:
    layer.trainable = False

In [12]:
# Adding custom layers on top of InceptionV3
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # New FC layer, random init
predictions = Dense(1, activation='sigmoid')(x)  # New softmax layer
model = Model(inputs=base_model.input, outputs=predictions)


In [13]:
mlflow.set_experiment(experiment_id="322113877408650181")
 # set the experiment id from the mlflow website whenever new experiment is created 


<Experiment: artifact_location='mlflow-artifacts:/322113877408650181', creation_time=1708515694937, experiment_id='322113877408650181', last_update_time=1708515694937, lifecycle_stage='active', name='FREEZING_AND_NON_FREEZING_SIGNAL_IMAGES_PROJECT', tags={}>

In [14]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.tensorflow.autolog()
# set the experiment id

#Initial Training


with mlflow.start_run(run_name="cropped_changed_TO_WAVELET_DATA_with_batch_inceptionv3_without_finetuning"):
    # Additionally, use ModelCheckpoint to save the best model based on validation accuracy
     
    model_checkpoint = ModelCheckpoint('cropped_model_changed_inceptionv3.h5', monitor='val_accuracy', 
                                   mode='max', save_best_only=True, verbose=1)

    early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, verbose=1, 
                               restore_best_weights=True)

    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history_st = model.fit(train_generator, validation_data=validation_generator, epochs=50, callbacks=[model_checkpoint, early_stopping, tensorboard_callback])
    



2024/02/23 02:01:37 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.preprocessing.image.DirectoryIterator'>. Dataset logging skipped.
2024/02/23 02:01:37 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.preprocessing.image.DirectoryIterator'>. Dataset logging skipped.


Epoch 1/50
112/112 [==============================] - ETA: 0s - loss: 0.7689 - accuracy: 0.5359
Epoch 1: val_accuracy improved from -inf to 0.45500, saving model to cropped_model_changed_inceptionv3.h5
112/112 [==============================] - 82s 556ms/step - loss: 0.7689 - accuracy: 0.5359 - val_loss: 0.7031 - val_accuracy: 0.4550
Epoch 2/50
112/112 [==============================] - ETA: 0s - loss: 0.6927 - accuracy: 0.5414
Epoch 2: val_accuracy improved from 0.45500 to 0.50750, saving model to cropped_model_changed_inceptionv3.h5
112/112 [==============================] - 61s 538ms/step - loss: 0.6927 - accuracy: 0.5414 - val_loss: 0.6917 - val_accuracy: 0.5075
Epoch 3/50
112/112 [==============================] - ETA: 0s - loss: 0.6868 - accuracy: 0.5484
Epoch 3: val_accuracy did not improve from 0.50750
112/112 [==============================] - 58s 511ms/step - loss: 0.6868 - accuracy: 0.5484 - val_loss: 0.6962 - val_accuracy: 0.4975
Epoch 4/50
112/112 [========================

2024/02/23 02:39:54 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: '>=' not supported between instances of 'slice' and 'int'
2024/02/23 02:39:54 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\ravit\AppData\Local\Temp\tmpti5s2ckw\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\ravit\AppData\Local\Temp\tmpti5s2ckw\model\data\model\assets


In [ ]:
# Plotting training results
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history_st.history['accuracy'], label='Train')
plt.plot(history_st.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history_st.history['loss'], label='Train')
plt.plot(history_st.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

# Log the plot file to MLflow
mlflow.log_artifact("D://PROJECTS/ALL_PROJECTS/EYE-CLASSIFICATION/INCEPTION_TRAINING/output.png")

In [ ]:
from tensorflow.keras.models import load_model

# Load the fine-tuned model
loaded_model = load_model("D://PROJECTS/ALL_PROJECTS/EYE-CLASSIFICATION/INCEPTION_TRAINING/best_model_inceptionv3.h5")
# Assuming 'model' is your trained model for prediction and evaluation
predictions = loaded_model.predict(test_generator)
y_pred = np.where(predictions > 0.5, 1, 0).reshape(-1)  # Adjust threshold as needed
y_true = test_generator.classes



In [ ]:
# Calculate and print metrics
accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, predictions)

print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

# Logging metrics to MLflow
mlflow.log_metric("accuracy", accuracy)
mlflow.log_metric("recall", recall)
mlflow.log_metric("f1_score", f1)
mlflow.log_metric("roc_auc", roc_auc)



In [ ]:
# Classification report and confusion matrix
report = classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
print(report_df)
report_df.to_csv("classification_report.csv")
mlflow.log_artifact("classification_report.csv", artifact_path='metrics')
mlflow.log_artifact("best_model_inceptionv3.h5",artifact_path='model')

In [ ]:
cm = confusion_matrix(y_true, y_pred)
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt="d", ax=ax, cmap='Blues')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.savefig("confusion_matrix.png")
plt.show()

mlflow.log_artifact("confusion_matrix.png", artifact_path='metrics')

mlflow.end_run()